# Actividad práctica: Predicción de series de tiempo

En esta tarea se pide entrenar y evaluar un predictor para la serie de tiempo Mackey-Glass. Esta serie de tiempo se obtiene de la solución de la siguiente ecuación diferencial

$$
\frac{dy}{dt} = 0.2 \frac{ y(t-\tau)}{1 + y(t-\tau)^{10}} - 0.1 y(t),
$$

donde el parámetro $\tau$ controla el comportamiento dinámico de la serie de tiempo 

- Siga las instrucciones en este notebook para resolver el problema de predicción
- Conteste las preguntas que se encuentran en este enunciado
- Finalmente envíe su notebook con los resultados y respuestas a phuijse@inf.uach.cl
- No olvide cambiar el título para reflejar los integrantes de su grupo

In [ ]:
import numpy as np

%matplotlib notebook
from IPython.display import display, Audio, HTML
import matplotlib.pylab as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import scipy.signal
%matplotlib notebook
from matplotlib import animation, patches
import soundfile as sf
from style import *

import pandas as pd
from pandas import *

NMSE = lambda y, yhat : np.sum((y - yhat)**2)/np.var(y)

***

Use el código que se muestra a continuación para generar la serie de tiempo Mackey Glass

- Considere una razón señal a ruido (SNR) de 2.
- Considere $\tau=17$ (comportamiento debilmente caótico)

Se generaran 1000 muestras de la serie de tiempo. Use los primeros 500 puntos para entrenar y los siguientes 500 puntos para hacer predicción

In [ ]:

# número de observaciones (no modificar)
N = 1000
# Razón señal a ruido (2., 0.5)
SNR = 2.
# constantes de la ecuación diferencial (no modificar)
a, b = 0.1, 0.2
# comportamiento dinámico de Mackey-Glass (17, 30)
tau = 17.
print(tau)
# paso de integración (no modificar)
dt = 0.05
# condición incial (no modificar)
y0 = 0.9
# largo temporal (no modificar)
tt = 5.
t = np.linspace(0, tt, num=N)

N_full, tau_full = int(N*tt/dt), int(tau/dt)
ymg = y0*np.ones(shape=(N_full, ))
# Runge-Kutta integration
for n in range(tau_full, N_full-1):
    byd = b*ymg[n-tau_full]/(1.0 + ymg[n-tau_full]**10.0)
    yk1 = dt*(-a*ymg[n] + byd)
    yk2 = dt*(-a*(ymg[n]+yk1/2) + byd)
    yk3 = dt*(-a*(ymg[n]+yk2/2) + byd)
    yk4 = dt*(-a*(ymg[n]+yk3) + byd)
    ymg[n+1] = ymg[n] + yk1/6 + yk2/3 +yk3/3 +yk4/6;
ymg = ymg[::int(tt/dt)]
#ymg = ymg - np.mean(ymg) 
# Contaminación con ruido blanco aditivo
s_noise = np.sqrt(np.var(ymg)/SNR) 
np.random.seed(0)
y_obs = ymg + s_noise*np.random.randn(len(ymg))

# Gráfico
fig, ax = plt.subplots(1, figsize=(9, 3), tight_layout=True)
ax.plot(t[:500], y_obs[:500])
ax.set_title('Serie de tiempo Mackey-Glass (entrenamiento)');
    


***

1. Describa en detalle el algoritmo LMS indicando sus semejanzas y diferencias con el filtro de Wiener 
1. Partiendo del error instantaneo $J_n^s(\textbf{w}) = e_n^2$ derive la regla de actualización de pesos
1. La siguiente clase de *Python* predice y entrena un filtro LMS. Complete la línea que dice 

` self.w = ? `

con el valor correcto de actualización de peso del filtro LMS

 1) EL algoritmo LMS es un tipo de filtro adaptativo, usa el descenso del gradiente para estimar una señal variable en el tiempo. Este algoritmo es uno de los más utilizados usado debido a su baja complejidad computacional y estabilidad.

   El filtro consiste en dos componentes, la primera es el calculo de la salida del filtro FIR convulucionado la entrada con los coeficientes. Ademas de la estimación del error.  y la segunda componente es el coeficiente de actualizacion de pesos.
   
Semejanzas: 
            - LMS converge a la solucion de Wiener
            - LMS es derivado de Wiener
            - Son predictores/estimadores
            - Ambos puede ser usados para identificar la respuesta al impulso de sistema desconocido.
            - Se entrena de manera recursiva y online
            
Diferencias: 
             - LMS: No se requiere conocimiento estadístico del proceso
             - LMS: No es necesario calcular e invertir la matriz de correlación
             - LMS se actualiza online y tiene costo 𝐿.
             - LMS: El aprendizaje es estadístico
             - Wiener: se entrena offline y tiene costo 𝐿2
             - Wiener: El aprendizaje es determinista.


2) Derivada $J_n^s(\textbf{w}) = e_n^2$

Sabiendo que $e_n = (d_n - y_n)$,  tambien $y_n = \sum_{k=0}^{L} w_{n, k} u_{n-k}$
               

$$
\begin{align}
J^s_n(\textbf{w}) &= e_n^2  \\
&= (d_n - y_n)^2 \\
&= (d_n - \sum_{k=0}^{L} w_{n, k} u_{n-k} )^2 \\
\end{align}
$$

$$
\begin{align}
\frac{dJ(w)}{dw} &=   2 (d_n - \sum_{k=0}^{L} w_{n, k} u_{n-k})  -u_{n-k} \\
\frac{dJ(w)}{dw} &=   -2 e_n u_{n-k} \\
\end{align}
$$

Usando la regla SGD llegamos a:
$$
\begin{align}
\textbf{w}_{n+1} = \textbf{w}_{n} + 2 \mu e_n \textbf{u}_{n}\nonumber \\
\end{align}
$$


        

In [ ]:
class LMS_filter(object):
    
    def __init__(self, L=1, mu=0.5, normalized=True):
        self.L = L
        self.mu = mu
        self.w = np.zeros(shape=(L, ))
        self.normalized = normalized
    
    def __len__(self):
        return self.L
    
    def predict(self, u):
        return np.dot(self.w, u)
    
    def update(self, u, d):
        d_pred = self.predict(u)
        norm = 1.
        if self.normalized:
            norm = np.sum(u**2) + 1e-6
        self.w = self.w + 2 * self.mu * (d - d_pred) * u / norm

***
## Predicción con algoritmo LMS

1. Entrene el predictor con el algoritmo normalized LMS usando el siguiente bloque de código
1. Construya una tabla con los NMSE de entrenamiento y prueba para distintos valores de $\mu$ y $L$
    - Se recomienda hacer un barrido logarítmico en $\mu$ (por ejemplo `mu=np.logspace(-2, 0, num=20)`)
    - Use al menos los siguientes valores de $L$: [5, 10, 20, 30]
1. Describa cada experimento analizando sus resultados de forma cuantitativa y cualitativa
    - ¿Se sobreajuste el filtro a los datos de entrenamiento? 
    - ¿Se desestabiliza el filtro?
1. Indique que combinación obtiene menor MSE de prueba 
***

1. Repita el experimento para $\tau = 30$ (comportamiento fuertemente caótico) 
1. Compare los resultados obtenidos con cada serie de tiempo. ¿Qué casos son más sencillos y cuales más complicados?


3) Dada la cantidad de posibiladades que existen entre L y mu, revisaremos algunos casos mas relevantes:
    
    
Para 𝜏=17 (puede probar los valores, abajo se encuentra el gráfico con un slider)

a) L=10 y  mu=0.784760 Se puede apreciar en el grafico que esta sobreajustado, es decir, anda muy bien 
  para el entrenamiento, pero en el test se ve terriblemente afectado. Se desastebiliza bastante.
  Siendo MSE entrenamiento 1032.4427, prueba 5186.2362

b) L=20 y mu=0.29 Este es uno de los mejores casos, no esta sobreajustado, para el entrenamiento como con el test anda bastante bien,
ya que el error cuadratico es bastante pequeño. El filtro no se desastebiliza en general.
Siendo MSE entrenamiento 303.7687, prueba 311.9917.

c) L=30 y mu=0.61 Este es un caso medio, tiene un sobreajuste medio, posee un error cuadratico moderado,
                 Siendo MSE entrenamiento 377.1947, prueba 502.9953


4)
 Valores Optimos L = 30, mu = 0.088587, test = 187.915610



Para 𝜏=30

a) L=10 y  mu=0.784760 Se puede ver en el gráfico que se encuentra sobreajustado, tiene un error cuadratico tremendo, por lo que se desastebiliza bastante.
Siendo MSE entrenamiento 1519.2776, prueba 7629.6103

b) L=20 y mu=0.29 En este caso, se puede apreciar que esta mediamante sobreajustado, el error cuadratico es medio. Se desastebiliza medianamente. Siendo MSE entrenamiento 413.9651, prueba 869.1495

c) L=30 Y mu=0.61 Esta bastante sobreajustado, el error cuadratico es muy grande. Se desastebiliza bastante. Siendo MSE entrenamiento 395.9658, prueba 2181.1657.

4) Valores Optimos L = 20, mu = 0.069519, test = 316.133279

----------------------------------------------------------------------------------
Compare los resultados obtenidos con cada serie de tiempo. ¿Qué casos son más sencillos y cuales más complicados?

Se puede apreciar que para el 𝜏=30, en los casos anteriormentes estudiados, el resultado es mucho más caotico, por ejemplo para 𝜏=17 L=20 y mu=0.29, resulta ser uno de lo mejores casos, pero para el 𝜏=30, para ser un caso sin ningún atractivo. Tambien sucede para la mayoria de los otros casos, en vez de mejorar se empeoran más.


In [ ]:
data_NMSE_train = []
data_NMSE_test = []
L = [5,10,20,30]
mu = np.logspace(-2,0,num=20)

In [ ]:
def lms_update(L,mu,print_plot):
    lms = LMS_filter(L, mu, normalized=True)

    # Entrenamiento
    y_pred = np.zeros(shape=(len(y_obs), ))
    for k in range(lms.__len__(), 500):
        y_window = y_obs[k-lms.__len__():k]
        y_pred[k] = lms.predict(y_window)
        lms.update(d=y_obs[k], u=y_window)
    # Prueba
    for k in range(500, len(y_obs)):
        y_window = y_obs[k-lms.__len__():k]
        y_pred[k] = lms.predict(y_window)

    
    data_NMSE_train.append((NMSE(ymg[lms.__len__():500], y_pred[lms.__len__():500])))
    data_NMSE_test.append(NMSE(ymg[500:], y_pred[500:]))
                           
    if print_plot:
        print("MSE entrenamiento %0.4f, prueba %0.4f" %(NMSE(ymg[lms.__len__():500], y_pred[lms.__len__():500]), 
                                                NMSE(ymg[500:], y_pred[500:])))
    
        fig, ax = plt.subplots(3, figsize=(9, 6), tight_layout=True)
        ax[0].plot(t, y_obs, 'k.', alpha=0.5, label='Observado'); ax[0].legend();
        ax[1].plot(t, ymg, 'g-', alpha=0.5, lw=2, label='Intrínseco'); 
        ax[1].plot(t[:500], y_pred[:500], alpha=0.75, lw=2, label='Predicho train'); 
        ax[1].plot(t[500:], y_pred[500:], alpha=0.75, lw=2, label='Predicho test'); ax[1].legend();

        ax[2].plot(t[:500], (ymg[:500] - y_pred[:500])**2, label='Error cuadrático train'); 
        ax[2].plot(t[500:], (ymg[500:] - y_pred[500:])**2, label='Error cuadrático test'); ax[2].legend();

In [ ]:
L = [5,10,20,30]
mu = np.logspace(-2,0,num=20)

l_opt = mu_opt = test_opt = 100000000

for l_val in L:
    print("L = %i"%l_val)
    data_NMSE_train = []
    data_NMSE_test = []
    for mu_val in mu:
        lms_update(l_val,mu_val,print_plot=False)
    df = DataFrame({"MU":mu, "MSE TRAIN": data_NMSE_train,"MSE TEST": data_NMSE_test})
    print(df)
    
    for opt in data_NMSE_test:
        if opt < test_opt:
            test_opt = opt
            l_opt = l_val
            mu_opt = mu[data_NMSE_test.index(opt)]
    
    if l_val < 30 : print("--------------------------------------------------------------------------")
    
print("\n Valores Optimos L = %i, mu = %f, test = %f"%(l_opt,mu_opt,test_opt))

In [ ]:
def lms_slider(L,mu):
    lms_update(L,mu,print_plot=True)

print("Plot for TAU=%f"%tau)
interact(lms_slider, L=widgets.SelectionSlider(options=L, value=l_opt),mu=np.logspace(-2,0,num=20))

In [ ]:

# número de observaciones (no modificar)
N = 1000
# Razón señal a ruido (2., 0.5)
SNR = 2.
# constantes de la ecuación diferencial (no modificar)
a, b = 0.1, 0.2
# comportamiento dinámico de Mackey-Glass (17, 30)
tau = 30.
print("SERIES RESTARTED WITH TAU=%f"%tau)
# paso de integración (no modificar)
dt = 0.05
# condición incial (no modificar)
y0 = 0.9
# largo temporal (no modificar)
tt = 5.
t = np.linspace(0, tt, num=N)

N_full, tau_full = int(N*tt/dt), int(tau/dt)
ymg = y0*np.ones(shape=(N_full, ))
# Runge-Kutta integration
for n in range(tau_full, N_full-1):
    byd = b*ymg[n-tau_full]/(1.0 + ymg[n-tau_full]**10.0)
    yk1 = dt*(-a*ymg[n] + byd)
    yk2 = dt*(-a*(ymg[n]+yk1/2) + byd)
    yk3 = dt*(-a*(ymg[n]+yk2/2) + byd)
    yk4 = dt*(-a*(ymg[n]+yk3) + byd)
    ymg[n+1] = ymg[n] + yk1/6 + yk2/3 +yk3/3 +yk4/6;
ymg = ymg[::int(tt/dt)]
#ymg = ymg - np.mean(ymg) 
# Contaminación con ruido blanco aditivo
s_noise = np.sqrt(np.var(ymg)/SNR) 
np.random.seed(0)
y_obs = ymg + s_noise*np.random.randn(len(ymg))

In [ ]:
test_opt = 100000000

for l_val in L:
    print("L = %i"%l_val)
    data_NMSE_train = []
    data_NMSE_test = []
    for mu_val in mu:
        lms_update(l_val,mu_val,print_plot=False)
    df = DataFrame({"MU":mu, "MSE TRAIN": data_NMSE_train,"MSE TEST": data_NMSE_test})
    print(df)
    
    for opt in data_NMSE_test:
        if opt < test_opt:
            test_opt = opt
            l_opt = l_val
            mu_opt = mu[data_NMSE_test.index(opt)]
    
    if l_val < 30 : print("--------------------------------------------------------------------------")
    
print("\n Valores Optimos L = %i, mu = %f, test = %f"%(l_opt,mu_opt,test_opt))

In [ ]:
def lms_slider(L,mu):
    lms_update(L,mu,print_plot=True)

print("Plot for TAU=%f"%tau)
interact(lms_slider, L=widgets.SelectionSlider(options=L, value=l_opt),mu=np.logspace(-2,0,num=20))

***
## Predicción con algoritmo RLS

1. Describa en detalle el algoritmo RLS indicando sus semejanzas y diferencias con el algoritmo LMS
1. Partiendo del error histórico $J_N(\textbf{w}) = \sum_{i=1}^N \beta^{N-i} e_i^2$ derive la regla recursiva de actualización de pesos 
1. La siguiente clase de *Python* predice y entrena un filtro RLS. Complete las líneas que dice 

` self.w = ? ` y `self.Phi_inv = ?`

con el valor correcto de actualización de peso del filtro RLS

1. El  algoritmo RLS provee una mayor velocidad de convergencia y menor error que el algoritmo LMS.
Este algoritmo es una versión muy parecida del algoritmo LMS, pero en el criterio de optimización, 
es donde el algoritmo RLS, calcula una solución más exacta, es debido a esto que es más costoso (requiere muchas más operaciones).

Semejanzas: 
            - Son estimadores/predictores
            - Ambos son secuenciales, metodos online que resulven el mismo tipo de problema
            - Ambos convergen a la "misma" solución

Diferencias: 

             - RLS: Mayor velocidad de convergencia
             - RLS: Menos error
             - RLS: Mayor costo en operaciones 
             - RLS: Capaz de adaptarse a cambios más bruscos.
             
             - LMS: Menos costoso
             - LMS: Proceso Markov
                

***
2. Partiendo de
$$
\begin{align}
    J_N(\textbf{w}) = \sum_{i=1}^n \beta^{n-i} e_i^2, \text{donde } 0<\beta<=1
\end{align}
$$

El coste de la funcion se minimiza tomando la derivada parcial de $k$ de los coeficientes de $\textbf{w}_n$ e igualando a cero.

$$
\begin{align}
    \frac{\partial{J(\textbf{w}_n)}}{\partial \textbf{w}_n(k)} = \sum_{i=0}^n 2 \beta^{n-i}e_i\frac{\partial e_i}{\partial \textbf{w}_n (k)} = -\sum_{i=0}^n 2 \beta^{n-i}e_i \textbf{u}(i-k)=0
\end{align}
$$

Reemplazando $e_i$ por la definicion de la señal de error obtenemos.

$$
\begin{align}
\sum_{i=0}^n \beta^{N-i} \Big[ d_i - \sum_{k=0}^L \textbf{w}_{n,k} \textbf{u}_{i-k} \Big] d_{i-k} = 0
\end{align}
$$

Reordenando las ecuaciones.

$$
\begin{align}
\sum_{l=0}^L \textbf{w}_{n,k} \Big[ \sum_{i=0}^n \beta^{n-i} \textbf{u}_{i-l} \textbf{u}_{i-k} \Big] = \sum_{i=0}^n\beta^{n-i} d_i \textbf{u}_{i-k}
\end{align}
$$

Esta forma puede ser representada en terminos de matrices.

$$
\begin{align}
\Phi_n \textbf{w}_n = \theta_n
\end{align}
$$

Donde
- Matriz de correlacion ponderada. $\Phi_n = \beta \Phi_{n-1} + \textbf{u}_n \textbf{u}_n^T$
- Vector de correlacion cruzada ponderada. $\theta_n = \theta_{n-1} + d_n \textbf{u}_n$

Asi, podemos encontrar los coeficientes que minimizan la funcion como:

$$
\begin{align}
\textbf{w}_n = \Phi_n^{-1} \theta_n
\end{align}
$$

Queremos derivar una solucion recursiva de la forma

$$
\begin{align}
\textbf{w}_n = \textbf{w}_{n-1} - \Delta \textbf{w}_{n-1}
\end{align}
$$

Estableciendo las condiciones iniciales ($\Phi_0 = \delta I, \theta_0 = 0 $) y usando la identidad de Woodbury: $(A + UCV)^{-1} = A^{-1} - A^{-1} U (C^{-1} + VA^{-1} U)^{-1} V A^{-1}$

Con: $A = \Phi_{n-1}^{-1}, U = \textbf{u}_n, C = 1, V = \textbf{u}_n^{-1}$.
$$
\begin{align}
\Phi^{-1} &= \Big[ \beta \Phi_{n-1} + \textbf{u}_n \textbf{u}_n^T\Big]^{-1} \\
&= \beta^{-1} \Phi_{n-1}^{-1} - \beta^{-1} \textbf{k}_n \textbf{u}_n^T \Phi_{n-1}^{-1}
\end{align}
$$

Donde $\textbf{k}_n = \Phi^{-1}_n \textbf{u}_n$ es la ganancia.

Asi
$$
\begin{align}
\textbf{w}_n &= \Phi^{-1}_n \theta_n \\
&= \beta \Phi^{-1}_n \theta_{n-1} + d_n \Phi^{-1}_n \textbf{u}_n \\
&=  \Phi_{n-1}^{-1} \theta_{n-1} - \textbf{k}_n \textbf{u}_n^T \Phi_{n-1}^{-1} \theta_{n-1} + \Phi_n^{-1} \textbf{u}_n d_n ; \text{  si  }   \textbf{w}_{n-1} =  \Phi_{n-1}^{-1} \theta_{n-1} \\
&= \textbf{w}_{n-1} + \textbf{k}_n ( d_n - \textbf{u}_n^T  \textbf{w}_{n-1} ) \\
&=  \textbf{w}_{n-1} + \textbf{k}_n e_n
\end{align}
$$

Donde $\textbf{k}_n e_n = \Delta \textbf{w}_{n-1}$ tambien llamado Factor de correccion.

In [ ]:
class RLS_filter(object):
    
    def __init__(self, L=1, beta=0.9, delta=10.):
        self.L = L
        self.beta = beta
        self.w = np.zeros(shape=(L, ))
        self.Phi_inv = delta*np.eye(L)
    
    def __len__(self):
        return self.L
    
    def predict(self, u):
        return np.dot(self.w, u)
    
    def update(self, u, d):          
        invbeta = 1.0/self.beta
        d_pred = self.predict(u)
        e = d - d_pred
        r = 1. + invbeta*np.dot(np.dot(u, self.Phi_inv), u.T)
        k = invbeta*np.dot(self.Phi_inv, u)/r
        self.Phi_inv = invbeta* self.Phi_inv - invbeta * self.Phi_inv *np.sum(k*u)
        self.w = self.w + np.dot(k,e)
       
        

***

1. Entrene el predictor con el algoritmo RLS usando el siguiente bloque de código
1. Considere primero  $\tau=17$
1. Construya una tabla con los NMSE de entrenamiento y prueba para distintos valores de $\beta$ y $L$
    - Se recomienda hacer un barrido lineal en $\beta$ (por ejemplo `mu=np.linspace(0.8, 1.0, num=20)`)
    - Use al menos los siguientes valores de $L$: [5, 10, 20, 30]
1. Describa cada experimento analizando sus resultados de forma cuantitativa y cualitativa
    - ¿Cuánto demora el filtro en estabilizarse? 
    - ¿Se sobreajuste el filtro a los datos de entrenamiento? 
    - ¿Se desestabiliza el filtro?
1. Indique que combinación obtiene menor MSE de prueba 
1. Repita el experimento para $\tau=30$
1. Compare con los resultados obtenidos con el algoritmo LMS ¿Qué algoritmo demora menos en converger?


In [ ]:
# número de observaciones (no modificar)
N = 1000
# Razón señal a ruido (2., 0.5)
SNR = 2.
# constantes de la ecuación diferencial (no modificar)
a, b = 0.1, 0.2
# comportamiento dinámico de Mackey-Glass (17, 30)
tau = 17.
print("SERIES RESTARTED WITH TAU=%f"%tau)
# paso de integración (no modificar)
dt = 0.05
# condición incial (no modificar)
y0 = 0.9
# largo temporal (no modificar)
tt = 5.
t = np.linspace(0, tt, num=N)

N_full, tau_full = int(N*tt/dt), int(tau/dt)
ymg = y0*np.ones(shape=(N_full, ))
# Runge-Kutta integration
for n in range(tau_full, N_full-1):
    byd = b*ymg[n-tau_full]/(1.0 + ymg[n-tau_full]**10.0)
    yk1 = dt*(-a*ymg[n] + byd)
    yk2 = dt*(-a*(ymg[n]+yk1/2) + byd)
    yk3 = dt*(-a*(ymg[n]+yk2/2) + byd)
    yk4 = dt*(-a*(ymg[n]+yk3) + byd)
    ymg[n+1] = ymg[n] + yk1/6 + yk2/3 +yk3/3 +yk4/6;
ymg = ymg[::int(tt/dt)]
#ymg = ymg - np.mean(ymg) 
# Contaminación con ruido blanco aditivo
s_noise = np.sqrt(np.var(ymg)/SNR) 
np.random.seed(0)
y_obs = ymg + s_noise*np.random.randn(len(ymg))

In [ ]:
beta = np.linspace(0.8, 1.0, num=20)

data_NMSE_train = []
data_NMSE_test = []

In [ ]:
beta = np.linspace(0.8, 1.0, num=20)

def rls_update(L,beta,print_plot):
    NMSE = lambda y, yhat : np.sum((y - yhat)**2)/np.var(y)
    
    rls = RLS_filter(L, beta, delta=1.)
    # Entrenamiento
    y_pred = np.zeros(shape=(len(y_obs), ))
    for k in range(rls.__len__(), 500):
        y_window = y_obs[k-rls.__len__():k]
        rls.update(d=y_obs[k], u=y_window)
        y_pred[k] = rls.predict(y_window)
    # Prueba
    for k in range(500, len(y_obs)):
        y_window = y_obs[k-rls.__len__():k]
        y_pred[k] = rls.predict(y_window)
        
    data_NMSE_train.append(NMSE(ymg[rls.__len__():500], y_pred[rls.__len__():500]))
    data_NMSE_test.append(NMSE(ymg[500:], y_pred[500:]))

    if print_plot:
        print("MSE entrenamiento %0.6f, prueba %0.6f" %(NMSE(ymg[rls.__len__():500], y_pred[rls.__len__():500]), 
                                                        NMSE(ymg[500:], y_pred[500:])))
        fig, ax = plt.subplots(3, figsize=(9, 6), tight_layout=True)
        ax[0].plot(t, y_obs, 'k.', alpha=0.5, label='Observado'); ax[0].legend();
        ax[1].plot(t, ymg, 'g-', alpha=0.5, lw=2, label='Intrínseco'); 
        ax[1].plot(t[:500], y_pred[:500], alpha=0.75, lw=2, label='Predicho train'); 
        ax[1].plot(t[500:], y_pred[500:], alpha=0.75, lw=2, label='Predicho test'); ax[1].legend();

        ax[2].plot(t[:500], (ymg[:500] - y_pred[:500])**2, label='Error cuadrático train'); 
        ax[2].plot(t[500:], (ymg[500:] - y_pred[500:])**2, label='Error cuadrático test'); ax[2].legend(); 

        

In [ ]:
l_opt = beta_opt = test_opt = 100000000


for l_val in L:
    print("L = %i"%l_val)
    data_NMSE_train = []
    data_NMSE_test = []
    for beta_val in beta:
        rls_update(l_val,beta_val,print_plot=False)
    df = DataFrame({"BETA":beta, "MSE TRAIN": data_NMSE_train,"MSE TEST": data_NMSE_test})
    print(df)
    
    for opt in data_NMSE_test:
        if opt < test_opt:
            test_opt = opt
            l_opt = l_val
            beta_opt = beta[data_NMSE_test.index(opt)]
    
    if l_val < 30 : print("--------------------------------------------------------------------------")
    
print("\n Valores Optimos L = %i, BETA = %f, test = %f"%(l_opt,beta_opt,test_opt))

In [ ]:
def rls_slider(L,beta):
    rls_update(L,beta,print_plot=True)

print("Plot for TAU=%f"%tau)
interact(rls_slider, L=widgets.SelectionSlider(options=L, value=l_opt),beta = np.linspace(0.8, 1.0, num=20))

In [ ]:
# número de observaciones (no modificar)
N = 1000
# Razón señal a ruido (2., 0.5)
SNR = 2.
# constantes de la ecuación diferencial (no modificar)
a, b = 0.1, 0.2
# comportamiento dinámico de Mackey-Glass (17, 30)
tau = 30.
print("SERIES RESTARTED WITH TAU=%f"%tau)
# paso de integración (no modificar)
dt = 0.05
# condición incial (no modificar)
y0 = 0.9
# largo temporal (no modificar)
tt = 5.
t = np.linspace(0, tt, num=N)

N_full, tau_full = int(N*tt/dt), int(tau/dt)
ymg = y0*np.ones(shape=(N_full, ))
# Runge-Kutta integration
for n in range(tau_full, N_full-1):
    byd = b*ymg[n-tau_full]/(1.0 + ymg[n-tau_full]**10.0)
    yk1 = dt*(-a*ymg[n] + byd)
    yk2 = dt*(-a*(ymg[n]+yk1/2) + byd)
    yk3 = dt*(-a*(ymg[n]+yk2/2) + byd)
    yk4 = dt*(-a*(ymg[n]+yk3) + byd)
    ymg[n+1] = ymg[n] + yk1/6 + yk2/3 +yk3/3 +yk4/6;
ymg = ymg[::int(tt/dt)]
#ymg = ymg - np.mean(ymg) 
# Contaminación con ruido blanco aditivo
s_noise = np.sqrt(np.var(ymg)/SNR) 
np.random.seed(0)
y_obs = ymg + s_noise*np.random.randn(len(ymg))

In [ ]:
l_opt = beta_opt = test_opt = 100000000


for l_val in L:
    print("L = %i"%l_val)
    data_NMSE_train = []
    data_NMSE_test = []
    for beta_val in beta:
        rls_update(l_val,beta_val,print_plot=False)
    df = DataFrame({"BETA":beta, "MSE TRAIN": data_NMSE_train,"MSE TEST": data_NMSE_test})
    print(df)
    
    for opt in data_NMSE_test:
        if opt < test_opt:
            test_opt = opt
            l_opt = l_val
            beta_opt = beta[data_NMSE_test.index(opt)]
    
    if l_val < 30 : print("--------------------------------------------------------------------------")
    
print("\n Valores Optimos L = %i, BETA = %f, test = %f"%(l_opt,beta_opt,test_opt))

In [ ]:
def rls_slider(L,beta):
    rls_update(L,beta,print_plot=True)

print("Plot for TAU=%f"%tau)
interact(rls_slider, L=widgets.SelectionSlider(options=L, value=l_opt),beta = np.linspace(0.8, 1.0, num=20))

RLS
Para tau = 17

4.
- Observando los graficos comparativos del filtro RLS, se puede afirmar que la convergencia (entiendase decaimiento  y estabilizacion del error cuadratico) del filtro RLS es bastante rapida (t < 0.2) si miramos el error cuadratico de entrenamiento.
- Para ciertos valores de L y $\beta$ (por ejemplo, L=30 y $\beta$ > 0.9) se puede observar claramente el sobreajuste del filtro a los datos.
- Por lo observado en el grafico, el filtro no alcanza a desestabilizarse de forma notoria, independiente de los valores de L y $\beta$.
5.
A partir de los datos de la tabla para cada combinacion de L y $\beta$, el mejor MSE de prueba para tau=17. se obtiene con 
$L = 30, \beta = 0.821, test = 188.056$

RLS
Para tau = 30.

4.
- Para este valor de tau, la convergencia del filtro (entiendase decaimiento y estabilizacion del error cuadratico) es mucho mas rapida, observandose valores por debajo de 0.15[seg] (para L = 30, $\beta$ = 0.905263), sin embargo, para valores de L = 5 y $\beta$ < 1 el valor del error cuadratico medio cambia de manera muy rapida y su valor es en general ,comparativamente para este tau y L, el mas alto.
- Se puede observar claramente un sobreajuste para valores de L=20 y $\beta$ > 0.9. Este sobreajuste es bastante pronunciado, ya que la diferencia entre la seccion del grafico de prueba y el grafico intrinseco es bastante marcada.
- El filtro no se desestabiliza de forma notoria para ningun valor de L y $\beta$.
5.
Analizando los datos de la tabla para tau=30, L=[5,10,20,30] y $0.8 <= \beta <= 1$; el mejor MSE de prueba se obtiene con $L = 30, \beta = 0.905263$ y su valor es MSE Test = $308.8732$

7. Comparando los resultados obtenidos con los diferentes valores de tau, tanto para el filtro LMS como para el filtro RLS, el algoritmo que tiende a converger de manera mas rapida es el algoritmo RLS, aunque se requiera un gasto computacional mayor.